In [ ]:
default_f = "../data-sets/input_8_v2.bay"
dyn_params = ["agent.seed", "disease.p_trans"]

lines = []
with open(default_f) as fin:
    for line in fin:
        idx = line.find("=")
        if idx != -1:
            param = line[:idx].strip()
            if param not in dyn_params:
                lines.append(line)

with open("input_map.csv", "w") as mout:
    mout.write("fname,seed,p_trans\n")
    i = 1
    for seed in range(1, 51):
        for p_trans in [0.0001, 0.00015, 0.0002, 0.00025, 0.0003, 0.00035, 0.0004, 0.00045, 0.0005, 0.00055]:
            fname = f"input_8_v2_{i}.bay"
            mout.write(f"{fname},{seed},{p_trans}\n")
            with open(fname, "w") as iout:
                iout.writelines(lines)
                iout.write(f"agent.seed = {seed}\n")
                iout.write(f"disease.p_trans = {p_trans}\n")
            
            i += 1
